### Step 1. 导入必要的库

In [31]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import numpy as np

### Step 2. 定义列表和字典储存获取的信息

In [5]:
# 储存爬取的信息
movie_dict = {}

# 直接提取的字符串
movie_name = []      # 片名
movie_review = []    # 短评

# 需要处理的字符串
movie_descriptions = []    # 导演、主演、国家、年代、类型等
movie_rate = []            # 评分及人数

### Step 3. 爬虫函数

In [6]:
headers = {'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Mobile Safari/537.36'}

In [7]:
def spider_douban(url):
    '''输入url，返回所需信息'''
    response = requests.get(url, headers=headers, timeout=10)  # 加入headers，伪装成浏览器
    response.encoding = 'utf-8'      # 设置格式
    soup = BeautifulSoup(response.text, 'html.parser')    # 指定文档解析器为html.parser，同一篇文档被不同的解析器解析后可能会生成不同结构的树型文档

    aim_contents = soup.find_all('div', class_='item')    # 先找到所需信息所在的标签：由25部电影信息组成的列表

    for aim_content in aim_contents:                    # 需要string和get_text得到字符串信息 
        movie_name.append(aim_content.find('span').string) 
        movie_review.append(aim_content.find('span', {'class': 'inq'}).string) 
        movie_descriptions.append(aim_content.find('p').get_text(strip=True))
        movie_rate.append(aim_content.find('div', {'class': 'star'}).get_text(",", strip=True))

### Step 4. 开始爬虫

In [8]:
for i in range(0, 226, 25):
    url = "https://movie.douban.com/top250?start=%s&filter=" % i
    spider_douban(url)

### Step 5. 检验爬虫结果

In [27]:
len(movie_name), movie_name[0]

(250, '肖申克的救赎')

In [26]:
len(movie_review), movie_review[0]

(250, '希望让人自由。')

In [21]:
len(movie_descriptions), movie_descriptions[0]

(250,
 '导演: 弗兰克·德拉邦特 Frank Darabont\xa0\xa0\xa0主演: 蒂姆·罗宾斯 Tim Robbins /...1994\xa0/\xa0美国\xa0/\xa0犯罪 剧情')

In [75]:
len(movie_rate), movie_rate[0]

(250, '9.7,1746699人评价')

### Step 6. 储存为DataFrame

In [28]:
movie_dict['片名'] = movie_name
movie_dict['短评'] = movie_review
movie_dict['描述'] = movie_descriptions
movie_dict['评分及人数'] = movie_rate

In [76]:
df = pd.DataFrame(movie_dict)

### Step 7. 数据处理

In [77]:
df.index = np.arange(1, 251)
df.head()

,片名,短评,描述,评分及人数
1,肖申克的救赎,希望让人自由。,导演: 弗兰克·德拉邦特 Frank Darabont 主演: 蒂姆·罗宾斯 Tim R...,"9.7,1746699人评价"
2,霸王别姬,风华绝代。,导演: 陈凯歌 Kaige Chen 主演: 张国荣 Leslie Cheung / 张...,"9.6,1290699人评价"
3,阿甘正传,一部美国近现代史。,导演: 罗伯特·泽米吉斯 Robert Zemeckis 主演: 汤姆·汉克斯 Tom ...,"9.5,1346921人评价"
4,这个杀手不太冷,怪蜀黍和小萝莉不得不说的故事。,导演: 吕克·贝松 Luc Besson 主演: 让·雷诺 Jean Reno / 娜塔...,"9.4,1541793人评价"
5,美丽人生,最美的谎言。,导演: 罗伯托·贝尼尼 Roberto Benigni 主演: 罗伯托·贝尼尼 Robe...,"9.5,819858人评价"


#### Step 7.1 描述：“导演: 弗兰克·德拉邦特 Frank Darabont\xa0\xa0\xa0主演: 蒂姆·罗宾斯 Tim Robbins /...1994\xa0/\xa0美国\xa0/\xa0犯罪 剧情”

#### ①提取出导演

In [78]:
df['导演'] = df['描述'].apply(lambda x: re.search('导演: (\S+)', x).group(1))

#### ②提取出国家

In [79]:
df['国家'] = df['描述'].apply(lambda x: re.split('/', x)[-2].strip())  # '\xa0美国\xa0'

#### ③提取出类型

In [80]:
df['类型'] = df['描述'].apply(lambda x: re.split('/', x)[-1].strip())  # '\xa0犯罪 剧情'

#### ④提取出年代

In [81]:
df['年代'] = df['描述'].apply(lambda x: re.search('\d+', x).group())

In [82]:
df['年代'] = df['描述'].apply(lambda x: re.findall('\d+', x)[0])     # re.findall()返回的是列表

In [83]:
df['年代'] = df['描述'].str.extract('(\d+)')

#### Step 7.2 评分及人数：“9.7,1746699人评价”

#### ①提取出评分

In [84]:
df['评分'] = df['评分及人数'].apply(lambda x: x.split(',')[0])

#### ②提取出评价人数

In [85]:
df['人数'] = df['评分及人数'].apply(lambda x: re.search(',(\d+)', x).group(1))

#### Step 7.3 删除多余列，并调整顺序

In [86]:
df = df.drop(['描述', '评分及人数'], axis=1)

In [87]:
df = df.reindex(columns=['片名', '国家', '年代', '导演','评分','人数','类型','短评'])

### Step 8. 查看数据，并保存到本地

In [89]:
df.head(10)

,片名,国家,年代,导演,评分,人数,类型,短评
1,肖申克的救赎,美国,1994,弗兰克·德拉邦特,9.7,1746699,犯罪 剧情,希望让人自由。
2,霸王别姬,中国大陆 中国香港,1993,陈凯歌,9.6,1290699,剧情 爱情 同性,风华绝代。
3,阿甘正传,美国,1994,罗伯特·泽米吉斯,9.5,1346921,剧情 爱情,一部美国近现代史。
4,这个杀手不太冷,法国,1994,吕克·贝松,9.4,1541793,剧情 动作 犯罪,怪蜀黍和小萝莉不得不说的故事。
5,美丽人生,意大利,1997,罗伯托·贝尼尼,9.5,819858,剧情 喜剧 爱情 战争,最美的谎言。
6,泰坦尼克号,美国,1997,詹姆斯·卡梅隆,9.4,1284683,剧情 爱情 灾难,失去的才是永恒的。
7,千与千寻,日本,2001,宫崎骏,9.3,1375715,剧情 动画 奇幻,最好的宫崎骏，最好的久石让。
8,辛德勒的名单,美国,1993,史蒂文·斯皮尔伯格,9.5,692749,剧情 历史 战争,拯救一个人，就是拯救整个世界。
9,盗梦空间,美国 英国,2010,克里斯托弗·诺兰,9.3,1312274,剧情 科幻 悬疑 冒险,诺兰给了我们一场无法盗取的梦。
10,忠犬八公的故事,美国 英国,2009,莱塞·霍尔斯道姆,9.3,889947,剧情,永远都不能忘记你所爱的人。


In [93]:
df.to_csv('豆瓣电影Top250.csv', encoding='utf_8_sig')  # 需要指定编码格式，不然会乱码